In [ ]:
import networkx as nx # type: ignore
import random
import osmnx as ox # type: ignore
import matplotlib.pyplot as plt # type: ignore
from shapely.geometry import Point # type: ignore
from tqdm import tqdm # type: ignore

%run toy_city_generation.ipynb
%run city_utilities.ipynb
%run real_city_generation.ipynb

%pip install osmnx matplotlib

In [ ]:
# Get the area that a given trip could start and end in
# In essence, we're getting all points in the start zone and end zone
# that have a valid path to the start and end points of the trip respectively
def get_valid_area(zone_points, get_distance, trip):
    start_points = zone_points[trip["start_zone_id"]]
    end_points = zone_points[trip["end_zone_id"]]

    # A start point is valid if a path exists starting from it and ends in the end zone, that is the correct length
    valid_start_points = [
        point
        for point in start_points
        if any(
            abs(get_distance(point, end_point) - trip["distance"]) < 1e-6
            for end_point in end_points
        )
    ]

    valid_end_points = [
        point
        for point in end_points
        if any(
            abs(get_distance(start_point, point) - trip["distance"]) < 1e-6
            for start_point in start_points
        )
    ]
    
    print(len(start_points))
    print(len(end_points))

    fraction_start = len(valid_start_points) / len(start_points) if start_points else 0
    fraction_end = len(valid_end_points) / len(end_points) if end_points else 0
    return valid_start_points, valid_end_points, fraction_start, fraction_end

def readout_average_thinning_factor(zone_points, get_distance, trips):
    fraction_start_sum = sum(get_valid_area(zone_points, get_distance, trip)[2] for trip in trips)
    fraction_end_sum = sum(get_valid_area(zone_points, get_distance, trip)[3] for trip in trips)

    print(f"Average thinning factor: {fraction_start_sum / len(trips):.2f} for start and {fraction_end_sum / len(trips):.2f} for end")


In [ ]:
def crude_predict_start_end_points(zones, points_in_zone, street_graph, trip, time_per_unit, distance_lookup):
    start_zone = next((z for z in zones if z["id"] == trip["start_zone_id"]), None)
    end_zone = next((z for z in zones if z["id"] == trip["end_zone_id"]), None)

    start_point = random.choice(points_in_zone[start_zone["id"]])
    end_point = random.choice(points_in_zone[end_zone["id"]])

    route = nx.shortest_path(street_graph, start_point["id"], end_point["id"], weight="weight")
    distance = distance_lookup[start_point["id"]][end_point["id"]]
    distance = nx.shortest_path_length(street_graph, start_point["id"], end_point["id"], weight="weight")
    travel_time = distance / time_per_unit
    distance_error = abs(distance - trip["travel_time"])
    time_error = abs(travel_time - trip["travel_time"])

    for _ in range(500):
        new_start_point = random.choice(points_in_zone[start_zone["id"]])
        new_end_point = random.choice(points_in_zone[end_zone["id"]])

        try:
            new_route = nx.shortest_path(street_graph, new_start_point["id"], new_end_point["id"], weight="weight")
        except nx.NetworkXNoPath:
            continue
        new_distance = nx.shortest_path_length(street_graph, new_start_point["id"], new_end_point["id"], weight="weight")
        new_travel_time = new_distance / time_per_unit
        
        # Is the new route's travel time and distance closer to the target?
        new_distance_error = abs(new_distance - trip["distance"])
        new_time_error = abs(new_travel_time - trip["travel_time"])

        has_distance_improved = new_distance_error < distance_error and new_time_error <= time_error
        has_time_improved = new_time_error < time_error and new_distance_error <= distance_error
        
        if has_distance_improved or has_time_improved:
            print("Found a better route with distance error", new_distance_error, "and time error", new_time_error)
            route = new_route
            distance = new_distance
            travel_time = new_travel_time
            start_point = new_start_point
            end_point = new_end_point
            distance_error = new_distance_error
            time_error = new_time_error

    # Assign our inferred start and end points to the trip
    trip["estimated_start_point"] = start_point
    trip["estimated_end_point"] = end_point
    trip["estimated_route"] = route

In [ ]:
# Main function

USE_FAKE_CITY = False

if USE_FAKE_CITY:
    zones, street_graph, trips, width, height = generate_synthetic_city_data() # type: ignore
    visualise(street_graph, zones, width, height, trips=trips) # type: ignore
else:
    zones, street_graph, trips = generate_real_city_data() # type: ignore
    display_real_city(street_graph, zones, trips) # type: ignore

In [ ]:
def is_point_in_zone(point, zone_polygon):
    point_geom = Point(point)
    return point_geom.within(zone_polygon)

def get_street_points_by_zone(street_graph, zones):
    street_points_by_zone = {}
    for zone_id, zone_row in tqdm(zones.iterrows(), desc="Processing zones", total=len(zones)):
        zone_polygon = zone_row.geometry
        if zone_polygon.is_empty:
            print(f"Zone {zone_id} has an empty geometry.")
            continue

        street_points_by_zone[zone_id] = []
        for node_id, data in tqdm(street_graph.nodes(data=True), desc=f"Processing nodes for zone {zone_id}", leave=False):
            point = (data["x"], data["y"])
            if is_point_in_zone(point, zone_polygon):
                street_points_by_zone[zone_id].append({"x": data["x"], "y": data["y"], "id": node_id})
        
        if not street_points_by_zone[zone_id]:
            print(f"Zone {zone_id} has no points.")

    return street_points_by_zone

#street_points_by_zone = get_street_points_by_zone(street_graph, zones)

In [ ]:
def get_all_distances():
    nodes = list(street_graph.nodes)
    progress_bar = tqdm(total=len(nodes), desc="Calculating distances")
    distance_lookup = {}
    for node, lengths in nx.all_pairs_dijkstra_path_length(street_graph, weight="weight"):
        distance_lookup[node] = lengths
        progress_bar.update(1)
    progress_bar.close()
    return distance_lookup

#distance_lookup = get_all_distances()
